<a id='import-scope-calculation'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Import Scopes (IFRS 17 Methodology Business Logic)</span> </p>

This notebook contains the logic used to perform calculations upon import of data (actuals and cash flows). This is also called 'Data Loading' and the concept of Scope is used here to define the logic and provide the means of executing the logic.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

## Notebooks

In [0]:
#!import "ImportStorage"

<a id='scopes'></a>
# Scopes
Scopes are used to define and perform data handling in a structured and easy-to-read-through fashion.

The <code>IModel</code> interface below will be used to execute calculations (i.e. evaluate the scopes) based on imported data.

In [0]:
public interface IModel : IMutableScopeWithStorage<ImportStorage>{}

## Getting Existing Identities

An [Identity](../DataModel/DataStructure#import-identity) is a set of identifiers for a certain set of data. In particular, an identity consists of a certain [AoC Type](../DataModel/DataStructure#aoc-type), [Novelty](../DataModel/DataStructure#novelty), [Data Node](../DataModel/DataStructure#data-node), Accident Year, and information on whether the data is for reinsurance or not. 

Given a certain Data Node and Accident Year, the interface <code>GetIdentities</code> returns all the existing identities (e.g. for Actuals and Cash flows) which have that Data Node and Accident Year.

In [0]:
public interface AllCfIdentities : IScope<string, ImportStorage> // string represents a DataNode
{
    public IEnumerable<ImportIdentity> ids => GetStorage().GetAllAocSteps(InputSource.Cashflow)
                                                          .Select(aocStep => new ImportIdentity {
                                                              AocType = aocStep.AocType,
                                                              Novelty = aocStep.Novelty,
                                                              DataNode = Identity
                                                              });
}

In [0]:
public interface GetIdentities : IScope<string, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<GetIdentities>(s => s.WithApplicability<AllCashflowIdentities>(x => x.GetStorage().IsSecondaryScope(x.Identity))
                                              .WithApplicability<GetActualIdentities>(x => x.GetStorage().ImportFormat == ImportFormats.Actual)
                                              .WithApplicability<GetCashflowIdentities>(x => x.GetStorage().ImportFormat == ImportFormats.Cashflow)
                                              .WithApplicability<GetAllIdentities>(x => x.GetStorage().ImportFormat == ImportFormats.Opening)
                                        );
    
    private IEnumerable<ImportIdentity> computedIdentities => new string[]{AocTypes.EA, AocTypes.AM, AocTypes.EOP}
                                                                .Select(aocType => new ImportIdentity {
                                                                AocType = aocType,
                                                                Novelty = Novelties.C,
                                                                DataNode = Identity
                                                                });
    private IEnumerable<ImportIdentity> allIdentities => ParsedIdentities.Concat(computedIdentities).Concat(SpecialIdentities).ToHashSet();                                                                                                   
                                                                                                       
    IEnumerable<ImportIdentity> ParsedIdentities => Enumerable.Empty<ImportIdentity>();    
    IEnumerable<ImportIdentity> SpecialIdentities => Enumerable.Empty<ImportIdentity>();
    
    //Set DataNode properties and ProjectionPeriod
    IEnumerable<ImportIdentity> Identities => allIdentities.Select(id => id with { IsReinsurance = GetStorage().DataNodeDataBySystemName[id.DataNode].IsReinsurance,
                                                                                   ValuationApproach = GetStorage().DataNodeDataBySystemName[id.DataNode].ValuationApproach
                                                                                 });
                                                          /* .SelectMany(id => Enumerable.Range(0,GetStorage().GetProjectionCount() + 1)
                                                                               .Select(pp => id with {ProjectionPeriod = pp })
                                                                       );*/
}

public interface AllCashflowIdentities : GetIdentities
{
    IEnumerable<ImportIdentity> GetIdentities.SpecialIdentities => GetScope<AllCfIdentities>(Identity).ids;
}


public interface GetActualIdentities : GetIdentities
{
    private IEnumerable<string> actualEstimateTypes => GetStorage().EstimateTypesByImportFormat[ImportFormats.Actual];
                                                                                                   
    IEnumerable<ImportIdentity> GetIdentities.ParsedIdentities => GetStorage().GetIfrsVariables(Identity).Where(iv => actualEstimateTypes.Contains(iv.EstimateType)).Select(v => new ImportIdentity(v));
    IEnumerable<ImportIdentity> GetIdentities.SpecialIdentities => GetScope<AllCfIdentities>(Identity).ids
                                                                                            .Concat(GetStorage().GetAllAocSteps(InputSource.Opening)
                                                                                              .Select(aocStep => new ImportIdentity {AocType = aocStep.AocType, 
                                                                                                 Novelty = aocStep.Novelty,
                                                                                                 DataNode = Identity
                                                                                                 }));
}

public interface GetCashflowIdentities : GetIdentities
{
    private bool isReinsurance => GetStorage().DataNodeDataBySystemName[Identity].IsReinsurance; //clean up in the next PR
    
    IEnumerable<ImportIdentity> GetIdentities.ParsedIdentities => GetStorage().GetRawVariables(Identity).Select(v => new ImportIdentity(v));
    
    IEnumerable<ImportIdentity> GetIdentities.SpecialIdentities => ParsedIdentities.Where(id => id.Novelty != Novelties.C)
                                                                                   .Select(id => id.Novelty).ToHashSet()
                                                                                   .SelectMany(n => (n == Novelties.N 
                                                                                   ? new string[]{AocTypes.IA, AocTypes.CF} //Add IA, CF, for New Business
                                                                                   : isReinsurance 
                                                                                       ? new string[]{AocTypes.IA, AocTypes.CF, AocTypes.YCU, AocTypes.CRU, AocTypes.RCU} //Add IA, CF, YCU, CRU, RCU for in force
                                                                                       : new string[]{AocTypes.IA, AocTypes.CF, AocTypes.YCU}) //Add IA, CF, YCU,
                                                                                   .Select(aocType => new ImportIdentity {
                                                                                   AocType = aocType,
                                                                                   Novelty = n,
                                                                                   DataNode = Identity }))
                                                                                   .Concat(new ImportIdentity {
                                                                                   AocType = AocTypes.CF, //Add CF for Deferral
                                                                                   Novelty = Novelties.C,
                                                                                   DataNode = Identity
                                                                                   }.RepeatOnce())   
                                                                                    .Concat(GetStorage().GetAllAocSteps(InputSource.Opening)
                                                                                              .Select(aocStep => new ImportIdentity {AocType = aocStep.AocType, 
                                                                                                 Novelty = aocStep.Novelty,
                                                                                                 DataNode = Identity
                                                                                                 }));   
}

public interface GetAllIdentities : GetIdentities
{
    IEnumerable<ImportIdentity> GetIdentities.SpecialIdentities => GetScope<AllCfIdentities>(Identity).ids
                                                                    .Concat(GetStorage().GetAllAocSteps(InputSource.Actual)
                                                                          .Select(aocStep => new ImportIdentity {AocType = aocStep.AocType, 
                                                                                                                 Novelty = aocStep.Novelty,
                                                                                                                 DataNode = Identity,
                                                                                                                 }));
}

## Getting Amount Types

Similarly, given a certain Data Node and Accident Year, the interface <code>ValidAmountType</code> returns all the amount types which are used in imported data with that Data Node and Accident Year.

In [0]:
public interface ValidAmountType : IScope<string, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
                builder.ForScope<ValidAmountType>(s => s.WithApplicability<BeAmountTypesFromIfrsVariables>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow ||
                                                                                                                x.GetStorage().IsSecondaryScope(x.Identity)));
    
    IEnumerable<string> BeAmountTypes => GetStorage().GetRawVariables(Identity)
                                            .Where(rv => rv.AmountType != null)
                                            .Select(x => x.AmountType)
                                            .Concat(GetStorage().DataNodeDataBySystemName[Identity].IsReinsurance ? (AmountTypes.CDR).RepeatOnce() : Enumerable.Empty<string>())
                                            .ToHashSet();
    
    
    IEnumerable<string> ActualAmountTypes => GetStorage().GetIfrsVariables(Identity)
                                                         .Where(iv => GetStorage().EstimateTypesByImportFormat[ImportFormats.Actual].Contains(iv.EstimateType))
                                                         .Select(x => x.AmountType)
                                                         .ToHashSet();
}
public interface BeAmountTypesFromIfrsVariables : ValidAmountType
{
    IEnumerable<string> ValidAmountType.BeAmountTypes => GetStorage().GetIfrsVariables(Identity)
                                                         .Where(iv => GetStorage().EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(iv.EstimateType) && iv.AmountType != null)
                                                         .Select(x => x.AmountType)
                                                         .ToHashSet();
}

<a id='aoc-step-structure'></a>
## AoC Step Structure

The AoC Step structure is constructed from the data which is delivered as input. It is assumed that it depends only on the Group of Contrat (i.e. it is invariant across Amount types or Accident Year). 

### Parent AoC Step 
The **ParentAocStep** scope computes and provides an important piece of information for each [Identity](../DataModel/DataStructure#import-identity), i.e. for a certain [AoC Type](../DataModel/DataStructure#aoc-type) and [Novelty](../DataModel/DataStructure#novelty).

It provides the list of the adjacent AoC Steps prior to Identity one. It can be more than one only for the step **CL** where a parent for each novelty is considered.
ParentAocStep is critical when computing the *telescoping* differences. 

They are defined as follows:

$$
\text{ParentAocStep}(\text{AoC Step}) = \left\{
\begin{array}{cl}
\text{AoC Step with AoC Type YCU and Novelty I} & \text{if AoC Type CRU} \\
\text{The last AoC Step with Data Type != Calculated and same Novelty as the AoC Step} & \text{if AoC Type YCU} \\
\text{The AoC Step which comes before in terms of order (as defined by AoC Type)} & \text{otherwise} \\
\end{array}
\right.
$$


In [0]:
public interface ParentAocStep : IScope<(ImportIdentity Id, string AmountType), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
                builder.ForScope<ParentAocStep>(s => s.WithApplicability<ParentAocStepForCreditRisk>(x => x.Identity.AmountType != AmountTypes.CDR));
    
    private HashSet<AocStep> ParsedAocSteps => GetScope<GetIdentities>(Identity.Id.DataNode).ParsedIdentities.Select(id => new AocStep(id.AocType, id.Novelty)).ToHashSet();
    private IEnumerable<AocStep> OrderedParsedAocSteps => ParsedAocSteps.Concat(CalculatedTelescopicAocStep).OrderBy(x => GetStorage().AocConfigurationByAocStep[x].Order);
    
    private Dictionary<AocStep,IEnumerable<AocStep>> ParentParsedIdentities => GetPreviousIdentities(OrderedParsedAocSteps);
    private AocStep identityAocStep => new AocStep(Identity.Id.AocType, Identity.Id.Novelty);
    
    IEnumerable<AocStep> CalculatedTelescopicAocStep => GetStorage().GetCalculatedTelescopicAocSteps();
    
    IEnumerable<AocStep> Values => 
        Identity.Id.AocType switch {
                 AocTypes.CRU => new AocStep(AocTypes.YCU, Novelties.I).RepeatOnce(),
                 AocTypes.YCU => OrderedParsedAocSteps.GetReferenceAocStepForCalculated(GetStorage().AocConfigurationByAocStep, identityAocStep).RepeatOnce(),
                 _ => ParentParsedIdentities.TryGetValue(identityAocStep, out var parents) ? parents : Enumerable.Empty<AocStep>(),
    };
}

public interface ParentAocStepForCreditRisk : ParentAocStep
{
   IEnumerable<AocStep> ParentAocStep.CalculatedTelescopicAocStep => 
       GetStorage().GetCalculatedTelescopicAocSteps().Where(aoc => aoc.AocType != AocTypes.CRU);
}

### Reference AoC Step

The **ReferenceAocStep** scope provides the AoC Step from which to retrieve the data in order to compute its value (e.g. AoC Step EA retrieves Present Values, while AoC Step YCU retrieves Nominal).

They are defined as follows:

$$
\text{ReferenceAocStep}(\text{AoC Step}) = \left\{
\begin{array}{cl}
\text{self} & \text{if AoC Step InputSource is not Calculated} \\
\text{The last AoC Step with Data Type != Calculated and same Novelty as the input AoC Step} ~, & \text{if AoC Type } \in \text{\{RCU, CF, IA, YCU, CRU\}} \\
\text{AoC Step with AoC Type CF and Novelty as the AoC Step} & \text{if AoC Type EA} \\
\text{AoC Step with AoC Type CL and Novelty C} & \text{if AoC Type $\in$ \{AM, EOP\}} \\
\text{empty} & \text{if AoC Type is BOP} \\
\text{log NotSupportedAocStepReference error} & \text{otherwise} \\
\end{array}
\right.
$$

where the last AoC Step is obtained by ordering the AoC Steps according to their order (as defined by its AoC Type) and taking the last one.

In [0]:
public interface ReferenceAocStep : IScope<ImportIdentity, ImportStorage> 
{
    private IEnumerable<AocStep> OrderedParsedAocSteps => GetScope<GetIdentities>(Identity.DataNode).ParsedIdentities
                                                                 .Select(id => new AocStep(id.AocType, id.Novelty))
                                                                 .Distinct()
                                                                 .OrderBy(aocStep => GetStorage().AocConfigurationByAocStep[aocStep].Order);
    private AocStep identityAocStep => new AocStep(Identity.AocType, Identity.Novelty);
    private AocStep GetReferenceAocStep(string aocType) {
        return aocType switch {
               AocTypes.RCU or AocTypes.CF or AocTypes.IA or AocTypes.YCU or AocTypes.CRU => OrderedParsedAocSteps.GetReferenceAocStepForCalculated(GetStorage().AocConfigurationByAocStep, identityAocStep),
               AocTypes.EA => new AocStep(AocTypes.CF, Identity.Novelty),
               AocTypes.AM or AocTypes.EOP => new AocStep(AocTypes.CL, Novelties.C),
               AocTypes.BOP => new AocStep(default, default), //BOP, C has DataType == Calculated. See ReferenceAocStep condition.
               _ => (AocStep)ApplicationMessage.Log(Error.NotSupportedAocStepReference, Identity.AocType),
        };
    }

    // The Reference AocStep from which get data (Nominal or PV) to compute
    AocStep Value => GetStorage().AocConfigurationByAocStep[identityAocStep].DataType == DataType.Calculated 
                     || GetStorage().AocConfigurationByAocStep[identityAocStep].DataType == DataType.CalculatedTelescopic 
                                                ? GetReferenceAocStep(Identity.AocType)
                                                : identityAocStep;
}

### Previous AoC Steps
The **PreviousAocSteps** scope computes and provides an other important piece of information for each [Identity](../DataModel/DataStructure#import-identity), i.e. for a certain [AoC Type](../DataModel/DataStructure#aoc-type) and [Novelty](../DataModel/DataStructure#novelty).

It provides the list of all previous AoC Steps up to the **BOP** step, whereby a Combined novelty will branch into the InForce and  New Business AoC Types.

PreviousAocSteps is critical when computing aggregated values along the various dimensions (such as for example Line of Business) and 
is formed by the ParentAocStep and its parent and so on until there is no parent.

$$
\text{PreviousAocSteps}(\rm{AocStep}) = \{PAS_1, PAS_2, \ldots\}
$$
where
$$
\rm{PAS}_1 = \rm{ParentAocStep}(\rm{AoC Step})
$$
$$
\rm{PAS}_2 = \rm{ParentAocStep}(\rm{PAS}_1).
$$

This scope depends on the InputSource (Actual or Cashflow) for which the PreviousSteps are requested due to the AocChain differences between Actual reports and the rest.

In [0]:
public interface PreviousAocSteps : IScope<(ImportIdentity Id, InputSource ScopeInputSource), ImportStorage> 
{   
    private AocStep identityAocStep => new AocStep(Identity.Id.AocType, Identity.Id.Novelty);
    private int aocStepOrder => GetStorage().AocConfigurationByAocStep[identityAocStep].Order;
    private HashSet<AocStep> allAocSteps => GetStorage().GetAllAocSteps(Identity.ScopeInputSource).ToHashSet();
    IEnumerable<AocStep> Values => allAocSteps.Contains(identityAocStep)
                                        ? GetScope<GetIdentities>(Identity.Id.DataNode).Identities
                                                                 .Select(id => new AocStep(id.AocType, id.Novelty))
                                                                 .Distinct()
                                                                 .Where(aoc => allAocSteps.Contains(aoc) && 
                                                                               GetStorage().AocConfigurationByAocStep[aoc].Order < aocStepOrder && 
                                                                               (Identity.Id.Novelty != Novelties.C ? aoc.Novelty == Identity.Id.Novelty : true) )
                                                                 .OrderBy(aoc => GetStorage().AocConfigurationByAocStep[aoc].Order)
                                        : Enumerable.Empty<AocStep>();
} 

The exact structure being return depends on the **order** of the AoC Steps (which is set by the [AoC Type](../DataModel/DataStructure#aoc-type)), and on which AoC steps exist.

<a id='discounting-calculation'></a>
## Discounting

The calculation of IFRS 17 figures is based on cumulated discounted cash flows.

<a id='yield-curves'></a>
### Yield Curves

The Yield Curves used for the discounting calculations are functions of the [Currency](../DataModel/DataStructure#currency), the [Data Node](../DataModel/DataStructure#data-node) and the [Economic Basis](../DataModel/DataStructure#economic-basis).

In particular:
- For the **Locked-in** economic basis, the yield curve used is the latest available as per end of the DataNode's inception year;
- Whereas for the **Current** economic base, the yield curve used is the latest available as per the current period.

The algorithm which retrieves the latest available yield curve is [here](../Utils/Queries#yield-curve).

<a id='interest-and-discount-rates'></a>
### Interest and Discount Rates and Factors

The factors used for discounting have the same granularity as the cash flow, i.e. monthly. The yield curves have yearly granularity, so the annual Interest factor is 1 + interest rate. The monthly Interest Interest and Discount factors are obtained from the annual factors such that the product of 12 months results in the annual factors, as follows:

$$
\text{Discount}_i = ( 1 + \text{YC}_i ) ^{-\frac{1}{12}} ~,
$$
and 
$$
\text{Interest}_i = ( 1 + \text{YC}_i) ^{\frac{1}{12}} ~,
$$

where the index $i$ denotes years.

In [0]:
public interface MonthlyRate : IScope<ImportIdentity, ImportStorage>
{
    private string EconomicBasis => GetContext();
    
    private double[] YearlyYieldCurve => GetStorage().GetYearlyYieldCurve(Identity, EconomicBasis);
    
    private double Perturbation => 0; //GetStorage().GetYieldCurvePerturbation(); => switch Args.Scenario { 10ptsU => 0.1, 10ptsD => -0.1, _ => default)
    
    double[] Interest => YearlyYieldCurve.Select(rate => Math.Pow(1d + rate, 1d / 12d) + Perturbation).ToArray();   
                        
    double[] Discount => Interest.Select(x => Math.Pow(x, -1)).ToArray();
}

<a id='nominal-values'></a>
## Nominal Values

The nominal cash flow values correspond to the cash flows provided in the cash flow input file. 
<br> These values are stored in the database as [RawVariable](../DataModel/DataStructure#raw-variables).
<br> Refer to the ReferenceAocStep of the AocStructure calculation to identify the correct AoC Type and Novelty to retrieve.
<br> Due to the Credit Default Risk of a reinsurance partner, the logic to compute the Nominal Cash flows for this Amount Type must be defined separately.

In [0]:
public interface NominalCashflow : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<NominalCashflow>(s => s.WithApplicability<CreditDefaultRiskNominalCashflow>(x => x.Identity.AmountType == AmountTypes.CDR && x.Identity.Id.AocType == AocTypes.CF)
                                                .WithApplicability<AllClaimsCashflow>(x => x.Identity.AmountType == AmountTypes.CDR));

    AocStep referenceAocStep => GetScope<ReferenceAocStep>(Identity.Id).Value;
    double[] Values => GetStorage().GetValues(Identity.Id with {AocType = referenceAocStep.AocType, Novelty = referenceAocStep.Novelty}, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear);
}

public interface CreditDefaultRiskNominalCashflow : NominalCashflow
{
    private double[] NominalClaimsCashflow => GetStorage().GetClaims()
                            .Select(claim => GetStorage().GetValues(Identity.Id with {AocType = referenceAocStep.AocType, Novelty = referenceAocStep.Novelty}, claim, Identity.EstimateType, Identity.AccidentYear))
                            .AggregateDoubleArray();
                            
    private double nonPerformanceRiskRate => GetStorage().GetNonPerformanceRiskRate(Identity.Id);
                            
    private double[] PvCdrDecumulated { get {
        var ret = new double[NominalClaimsCashflow.Length];
        for (var i = NominalClaimsCashflow.Length - 1; i >= 0; i--)
            ret[i] = Math.Exp(-nonPerformanceRiskRate) * ret.ElementAtOrDefault(i + 1) + NominalClaimsCashflow[i] - NominalClaimsCashflow.ElementAtOrDefault(i + 1);
        return ret; } } 
        
    double[] NominalCashflow.Values => Subtract(PvCdrDecumulated, NominalClaimsCashflow);
}

public interface AllClaimsCashflow : NominalCashflow
{
    double[] NominalCashflow.Values => GetStorage().GetClaims()
                                        .Select(claim => GetStorage().GetValues(Identity.Id with {AocType = referenceAocStep.AocType, Novelty = referenceAocStep.Novelty}, claim, Identity.EstimateType, Identity.AccidentYear))
                                        .AggregateDoubleArray();
}

For a given month $i$ they are denoted as $\rm{Nominal}_i$.

<a id='present-values'></a>
## Present Values

Present Values are calculated during the import of the cash flows and stored on the database. They are computed for the relevant Economic Basis, depending on the Valuation Basis.

Their calculation is described in the following sections and is summarized in the $\rm{PV}$ formula [below](#present-value).

<a id='cumulated-discounted-cash-flows'></a>
### Cumulated Discounted Cash flows

Cumulated and Discounted cash flows $\rm{CDC}$ are computed using the monthly discount rates and in a recursive manner, as follows:

$$
\text{CDC}_i(\text{AoC step}) = \left\{
\begin{array}{cl}
\text{Nominal}_i + \text{CDC}_{i+1} \cdot {\text{Valid Discount}_{\frac{i}{12}}} ~, & \text{if Amount Type's Period Type is Beginning Of Period} \\
\big( \text{Nominal}_i + \text{CDC}_{i+1} \big) \cdot {\text{Valid Discount}_{\frac{i}{12}}} ~, & \text{if Amount Type's Period Type is End Of Period}
\end{array}
\right.
$$

where Transaction Period depends on which Best Estimate value is being computed, in particular on what its [Amount Type](../DataModel/DataStructure#amount-type) is (each Amount Type has its own [Period Type](../Constants/Enums)); and $\text{Valid Discount}$ stands for the fact that in case the Discount Curves are shorter than the required index, then their last element is returned. We also need to flip the sign of the discounted and cumulated values, to create a reserve view and be consistent with the usual [Cash flow Sign Convention](https://en.wikipedia.org/wiki/Cash_flow_sign_convention). 

<br> Also here, the Credit Default Risk contribution is calculated separately. Since it is based on Claims, the Period Type is implicitly defined.
The risk of default is included in the discounting formula as follows. First, we notice that one can explicitly write the discounting recursive relation above as:
$$
\begin{array}{l}
\text{CDC}_i = \big( \text{Nominal}_i + \text{CDC}_{i+1} \big) \cdot {\text{Valid Discount}_{\frac{i}{12}}} 
\Rightarrow \text{CDC}_i = \sum_{\tau=i} \big( \text{Valid Discount}_{\frac{i}{12}} \big)^{\tau-i+1} \cdot \text{Nominal}_\tau 
\end{array}
$$

Thus, the cumulated and discounted cashflow correction for default risk is assigned to the Amount Type Credit Default Risk (CDR) and it is obtained from the Amount Type Claims multiplying the right hand side of the previous formula by the corresponding risk factor

$$
\text{CDC}_i^{\text{CDR}} = \sum_{\tau=i} \big( \text{Valid Discount}_{\frac{i}{12}} \big)^{\tau-i+1} \cdot \text{Nominal}_\tau^{\text{Claim}} \cdot \big( e^{-\gamma(\tau-i)} -1 \big)
$$

In [0]:
public interface DiscountedCashflow : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? Accidentyear), ImportStorage>
{
    private PeriodType periodType => GetStorage().GetPeriodType(Identity.AmountType, Identity.EstimateType); 
    
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<DiscountedCashflow>(s => s.WithApplicability<DiscountedCreditRiskCashflow>(x => x.Identity.Id.IsReinsurance && x.Identity.AmountType == AmountTypes.CDR));

    [NotVisible]
    string EconomicBasis => GetContext();

    [NotVisible]
    double[] MonthlyDiscounting => GetScope<MonthlyRate>(Identity.Id).Discount;
    
    [NotVisible]
    double[] NominalValues => GetScope<NominalCashflow>(Identity).Values;

    double[] Values => Multiply(-1d, NominalValues.ComputeDiscountAndCumulate(MonthlyDiscounting, periodType)); // we need to flip the sign to create a reserve view
}

public interface DiscountedCreditRiskCashflow : DiscountedCashflow
{     
    private double nonPerformanceRiskRate => GetStorage().GetNonPerformanceRiskRate(Identity.Id);
        
    double[] DiscountedCashflow.Values => Multiply(-1d, NominalValues.ComputeDiscountAndCumulateWithCreditDefaultRisk(MonthlyDiscounting, nonPerformanceRiskRate)); // we need to flip the sign to create a reserve view
}

<a id='telescopic-difference'></a>
### Telescoping Difference


Present Value figures for a specific period are typically reported through an analysis of change, where for each [AoC Step](#aoc-step-structure) the variation with respect to the preceding AoC Step is shown.

The Telescoping Difference is basically the delta between two adjacent AoC Steps, whereby the [ParentAocStep](#aoc-step-structure) is used to determine the AoC Step. 

It is defined as follows:

$$
\text{TelescopingDifference}_i = 
\text{CDC}_{i}\big(\text{current AoC Step}\big) - \text{CDC}_{i}\big(\text{parent AoC Step}\big)
$$

where AoC Type is the AoC Type of the AoC Step for which the calculations are being performed.

In [0]:
public interface TelescopicDifference : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? Accidentyear), ImportStorage>
{
    [NotVisible]
    string EconomicBasis => GetContext();
    private double[] CurrentValues => GetScope<DiscountedCashflow>(Identity).Values;
    
    private double[] PreviousValues => (GetScope<ParentAocStep>((Identity.Id, Identity.AmountType)))
                                            .Values
                                            .Select(aoc => GetScope<DiscountedCashflow>((Identity.Id with {AocType = aoc.AocType, Novelty = aoc.Novelty}, Identity.AmountType, Identity.EstimateType, Identity.Accidentyear)).Values)
                                            .Where(cf => cf.Count() > 0)
                                            .AggregateDoubleArray();
    
    double[] Values => Subtract(CurrentValues, PreviousValues);
}

<a id='present-value'></a>
### Present Value

The present value ($\rm{PV}$) can be determined by taking the appropriate elements of the cumulated discounted cash flows. This is done as function of the two [projection parameters](../DataModel/DataStructure#projection-configuration) $\rm{Shift}$ ($S$) and $\rm{TimeStep}$ ($TS$):

$$
\text{PV}(S, TS) = \left\{
\begin{array}{cl}
\text{PV}_{S}  ~, & \text{if Valuation Period is Beginning of Period} \\
\text{PV}_{S+TS/2 -1}  ~, & \text{if Valuation Period is Mid of Period} \\
\sum_{i=S}^{S + TS - 1}\text{PV }_{i}  ~, & \text{if Valuation Period is Delta} \\
\text{PV}_{S + TS}  ~, & \text{if Valuation Period is End of Period} \\
\end{array}
\right.
$$

where the term $TS/2$ uses MidpointRounding.AwayFromZero (as defined by *https:[]()//docs.microsoft.com/en-us/dotnet/api/system.midpointrounding?view=net-6.0)*: rounding to the nearest number, away from zero in the exact halfway case. Furthermore, if the array is smaller than the index, then the last element is returned.

For instance, for the current year and year-to-date view we have $S=0$ and $TS=3$ for the first quarter, $TS=6$ for the 2nd quarter and so on.
For the projection values of next year first quarter we would have $S=12$ and $TS=3$, etc.

In [0]:
public interface IWithGetValueFromValues : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    private int shift => GetStorage().GetShift(0);//Identity.Id.ProjectionPeriod
    private int timeStep => GetStorage().GetTimeStep(0);//Identity.Id.ProjectionPeriod
    
    public double GetValueFromValues(double[] Values)
    {
        return GetStorage().GetValuationPeriod(Identity.Id) switch {
                        ValuationPeriod.BeginningOfPeriod => Values.ElementAtOrDefault(shift),
                        ValuationPeriod.MidOfPeriod => Values.ElementAtOrDefault(shift + Convert.ToInt32(Math.Round(timeStep / 2d, MidpointRounding.AwayFromZero)) - 1),
                        ValuationPeriod.Delta => Values.Skip(shift).Take(timeStep).Sum(),
                        ValuationPeriod.EndOfPeriod  => Values.ElementAtOrDefault(shift + timeStep),
                        ValuationPeriod.NotApplicable => default
                    };
    }
}

<a id='interest-accretion'></a>
### Interest Accretion

Since the Interest Accretion cash flows are typically not provided as input (as they can be computed from its parent AoC Step), its present values can be computed as follows:

$$
\text{InterestAccretion}_i(\text{AoC step}) = \left\{
\begin{array}{cl}
\big(\text{CDC}_i(\text{Parent AoC step}) - \text{Nominal}_i(\text{parent AoC step}) \big) \cdot \big({\text{Valid Interest}_{\frac{i}{12}}} - 1 \big)~, & \text{if Amount Type's Transaction Period is Beginning of Period} \\
\text{CDC}_i(\text{parent AoC step}) \cdot \big({\text{Valid Interest}_{\frac{i}{12}}} - 1 \big)~, & \text{otherwise}
\end{array}
\right.
$$

<br> Due to the Credit Default Risk of a reinsurance partner, the logic to compute the Interest Accretion for this Amount Type must be defined separately. Since it is based on Claims, the Period Type is implicitly defined.

In [0]:
public interface IWithInterestAccretion : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    private double[] parentDiscountedValues => Multiply(-1d, GetScope<DiscountedCashflow>(Identity).Values);    
    private double[] parentNominalValues => GetScope<NominalCashflow>(Identity).Values;
    private double[] monthlyInterestFactor => GetScope<MonthlyRate>(Identity.Id).Interest;
    
    double[] GetInterestAccretion() 
    {
        var periodType = GetStorage().GetPeriodType(Identity.AmountType, Identity.EstimateType);
        var ret = new double[parentDiscountedValues.Length];
        
        switch (periodType) {
            case PeriodType.BeginningOfPeriod :
                for (var i = 0; i < parentDiscountedValues.Length; i++)
                     ret[i] = -1d * (parentDiscountedValues[i] - parentNominalValues[i]) * (GetElementOrDefault(monthlyInterestFactor, i/12) - 1d );
                break;
            default :
                for (var i = 0; i < parentDiscountedValues.Length; i++)
                     ret[i] = -1d * parentDiscountedValues[i] * (GetElementOrDefault(monthlyInterestFactor, i/12) - 1d );
             break;
        }
        
        return ret;
    }
}

public interface IWithInterestAccretionForCreditRisk : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    private double[] nominalClaimsCashflow => GetScope<AllClaimsCashflow>(Identity).Values;
    private double[] nominalValuesCreditRisk => Multiply(-1, GetScope<CreditDefaultRiskNominalCashflow>(Identity with {Id = Identity.Id with {AocType = AocTypes.CF}}).Values);
    private double[] monthlyInterestFactor => GetScope<MonthlyRate>(Identity.Id).Interest;
    private double nonPerformanceRiskRate => GetStorage().GetNonPerformanceRiskRate(Identity.Id);
    
    double[] GetInterestAccretion() 
    {
        var interestOnClaimsCashflow = new double[nominalClaimsCashflow.Length];
        var interestOnClaimsCashflowCreditRisk = new double[nominalClaimsCashflow.Length];
        var effectCreditRisk = new double[nominalClaimsCashflow.Length];
        for (var i = nominalClaimsCashflow.Length - 1; i >= 0; i--) {
            interestOnClaimsCashflow[i] = 1 / GetElementOrDefault(monthlyInterestFactor, i/12) * (interestOnClaimsCashflow.ElementAtOrDefault(i + 1) + nominalClaimsCashflow[i] - nominalClaimsCashflow.ElementAtOrDefault(i + 1));
            interestOnClaimsCashflowCreditRisk[i] = 1 / GetElementOrDefault(monthlyInterestFactor, i/12) * (Math.Exp(-nonPerformanceRiskRate) * interestOnClaimsCashflowCreditRisk.ElementAtOrDefault(i + 1) + nominalClaimsCashflow[i] - nominalClaimsCashflow.ElementAtOrDefault(i + 1));
            effectCreditRisk[i] = interestOnClaimsCashflow[i] - interestOnClaimsCashflowCreditRisk[i];
        }
            
        return Subtract(nominalValuesCreditRisk, effectCreditRisk);
    }
}

<a id='present-value'></a>
### Present Value

The PV values are valid for all choices of the [Economic Basis](../DataModel/DataStructure#economic-basis):

$$
\text{PV}_i (\text{AoC step}) = \left\{
\begin{array}{rl}
\text{CDC}_i ~, & \text{if AoC Type = BOP} \\
-\text{Nominal}_i(\text{Parent AoC Step}) ~, & \text{if AoC Type = CF } \\
\text{InterestAccretion}_i ~, & \text{if AoC Type = IA } \\
0 ~, & \text{if AoC Type = AM } \\\
\text{CDC}_i(\text{Parent AoC step}) ~, & \text{if AoC Type = EOP } \\
\text{TelescopingDifference}_i ~, & \text{otherwise}
\end{array}
\right.
$$

where $i$ denotes the months, and the [$\rm{TelescopingDifference}_i$](#telescopic-difference) and the [$\rm{InterestAccretion}_i$](#interest-accretion) quantities are defined above.

In [0]:
public interface PresentValue : IWithGetValueFromValues
{   
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<PresentValue>(s => s
                                        .WithApplicability<ComputePresentValueWithIfrsVariable>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow || x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode))
                                        .WithApplicability<PresentValueFromDiscountedCashflow>(x => (x.Identity.Id.AocType == AocTypes.BOP && x.Identity.Id.Novelty != Novelties.C) || x.Identity.Id.AocType == AocTypes.EOP)
                                        .WithApplicability<CashflowAocStep>(x => x.Identity.Id.AocType == AocTypes.CF)
                                        .WithApplicability<PresentValueWithInterestAccretionForCreditRisk>(x => x.Identity.Id.IsReinsurance && x.Identity.AmountType == AmountTypes.CDR && x.Identity.Id.AocType == AocTypes.IA)
                                        .WithApplicability<PresentValueWithInterestAccretion>(x => x.Identity.Id.AocType == AocTypes.IA)
                                        .WithApplicability<EmptyValuesAocStep>(x => new string[]{AocTypes.BOP, AocTypes.EA, AocTypes.AM, AocTypes.RCU}.Contains(x.Identity.Id.AocType) ) //add here combination CRU for At !CDR?
                                       );
    
    [NotVisible][IdentityProperty][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => GetContext();
    
    [NotVisible]
    double[] Values => GetScope<TelescopicDifference>(Identity).Values;
    
    public double Value => GetValueFromValues(Values);
}

public interface ComputePresentValueWithIfrsVariable : PresentValue
{
    double PresentValue.Value => GetStorage().GetValue(Identity.Id, Identity.AmountType, Identity.EstimateType, EconomicBasis, Identity.AccidentYear);
    double[] PresentValue.Values => Enumerable.Empty<double>().ToArray();
}

public interface PresentValueFromDiscountedCashflow : PresentValue
{
    [NotVisible]
    double[] PresentValue.Values => GetScope<DiscountedCashflow>(Identity).Values;
}

public interface CashflowAocStep : PresentValue
{
    [NotVisible]
    double[] PresentValue.Values => GetScope<NominalCashflow>(Identity).Values;
}

public interface PresentValueWithInterestAccretion : PresentValue, IWithInterestAccretion
{
    [NotVisible]
    double[] PresentValue.Values => GetInterestAccretion();
}

public interface PresentValueWithInterestAccretionForCreditRisk : PresentValue, IWithInterestAccretionForCreditRisk
{
    [NotVisible]
    double[] PresentValue.Values => GetInterestAccretion();
}

public interface EmptyValuesAocStep : PresentValue
{
    [NotVisible]
    double[] PresentValue.Values => Enumerable.Empty<double>().ToArray();
}

<a id='current-and-locked'></a>
### Current and Locked

PV Current and PV Locked below basically hold the Present Values [PV](#present-value) computed using the **Current** yield curves and the **Locked** yield curves, respectively.

Values are available for each AmountType (by calling <code>PvLocked.ByAmountType</code>):

$$
\text{PV Locked}(\text{AoC step}, \text{Amount Type}) = \text{PV}(\text{AoC step}, \text{Amount Type})|_{\text{Economic Base = L}}
$$

$$
\text{PV Current}(\text{AoC step}, \text{Amount Type}) = \text{PV}(\text{AoC step}, \text{Amount Type})|_{\text{Economic Base = C}}
$$


And aggregated values are also available as the sum over all [Amount Types](../DataModel/DataStructure#amount-type) (by calling <code>PvLocked.Value</code>):

$$
\text{PV Locked}(\text{AoC step}) = \sum_{\text{Amount Types}} \text{PV}(\text{AoC step}, \text{Amount Type})|_{\text{Economic Base = L}}
$$

$$
\text{PV Current}(\text{AoC step}) = \sum_{\text{Amount Types}} \text{PV}(\text{AoC step}, \text{Amount Type})|_{\text{Economic Base = C}}
$$

These are used in the BBA methodology, whereby in the CSM calculations only PV Locked is used, and both of them are stored in the database under the [IfrsVariable](../DataModel/DataStructure#ifrs-variable) data structure.

In [0]:
public interface PvLocked : IScope<ImportIdentity, ImportStorage>
{   
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.L;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.BE;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();  
    
    [NotVisible]
    PresentValue[] PresentValues => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes
                                        .SelectMany(at => accidentYears
                                            .Select(ay => GetScope<PresentValue>((Identity, at, EstimateType, ay), o => o.WithContext(EconomicBasis))))
                                            .ToArray();
    double Value => PresentValues.Aggregate().Value;
}

public interface PvCurrent : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.C;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.BE;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    PresentValue[] PresentValues => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes
                                        .SelectMany(at => accidentYears
                                            .Select(ay => GetScope<PresentValue>((Identity, at, EstimateType, ay), o => o.WithContext(EconomicBasis))))
                                            .ToArray();
    
    double Value => PresentValues.Aggregate().Value;
}

<a id='risk-adjustment'></a>
## Risk Adjustment

Risk Adjustment values ($\rm{RA}$) are accessible from the [PresentValue](#present-value) data and have [Estimate Type](../DataModel/DataStructure#estimate-type) $RA$. In particular, the Locked-In and Current values are given by:

$$
\text{RA Locked}(\text{AoC step}) = \text{PV}(\text{AoC step})|_{\text{Calculation Type = RA},~ \text{Economic Basis = L}}
$$

$$
\text{RA Current}(\text{AoC step}) = \text{PV}(\text{AoC step})|_{\text{Calculation Type = RA},~ \text{Economic Basis = C}}
$$

where PV is defined [above](#present-value) and uses the input cash flows with Calculation Type = RA.

In [0]:
public interface RaLocked : IScope<ImportIdentity, ImportStorage>
{   
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.L;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.RA;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();  
                                                               
    PresentValue[] PresentValues => accidentYears.Select(ay => GetScope<PresentValue>((Identity, (string)null, EstimateType, ay), o => o.WithContext(EconomicBasis))).ToArray();
    
    double Value => PresentValues.Aggregate().Value;
}
public interface RaCurrent : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EconomicBasis))]
    string EconomicBasis => EconomicBases.C;
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.RA;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();  
                                                               
    PresentValue[] PresentValues => accidentYears.Select(ay => GetScope<PresentValue>((Identity, (string)null, EstimateType, ay), o => o.WithContext(EconomicBasis))).ToArray(); 
    
    double Value => PresentValues.Aggregate().Value;
}

<a id='amortization'></a>
## Amortization

For the Amortization AoC Step (AoC Type **AM**), the amortization factors to be used are defined below.

<a id='coverage-units'></a>
### Coverage Units

The coverage unit (CU) of a GIC is introduced in the standard as the quantity of the service provided in that GIC. The service is
measured by considering the quantity of benefits provided as well as the expected coverage period of the GIC.

The cash flows of coverage units are retrieved from the discounted cash flows with [EstimateType](../DataModel/DataStructure#estimate-type) CU.

In [0]:
public interface CoverageUnitCashflow : IScope<ImportIdentity, ImportStorage>
{   
    [NotVisible] string EconomicBasis => GetContext();
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.CU;
    
    double[] Values => GetScope<DiscountedCashflow>((Identity, (string)null, EstimateType, (int?)null)).Values;
}

<a id='amortization-factor'></a>
### Amortization Factor

For a certain GIC, the monthly Amortization Factors $\text{Monthly }AF_i$ are computed from the cash flows of the underlying coverage unit for that GIC:

$$
\text{Monthly }AF_i = 1 - \frac{ \text{Nominal}_i(CL)} {\text{CDC}_i(CL) } ~.
$$

where:
- $i$ denotes a monthly period;
- the nominal cash flows $\text{Nominal}_i(CL)$ are the nominal cash flows of the coverage unit for the AoC Step **Combined Liability** (CL) (input data);
- and the corresponding cumulated discounted cash flows $\text{CDC}_i$ are defined [above](#cumulated-discounted-cash-flows).

In [0]:
public interface MonthlyAmortizationFactorCashflow : IScope<ImportIdentity, ImportStorage>
{
    private double[] NominalCuCashflow => GetScope<NominalCashflow>((Identity with {AocType = AocTypes.CL}, (string)null, EstimateTypes.CU, (int?)null)).Values;
    private double[] DiscountedCuCashflow => Multiply(-1d, GetScope<CoverageUnitCashflow>(Identity with {AocType = AocTypes.CL}, o => o.WithContext(EconomicBasis)).Values);
    
    [NotVisible] string EconomicBasis => GetContext();
    
    double[] MonthlyAmortizationFactors => Identity.AocType switch {
                AocTypes.AM => NominalCuCashflow.Zip(DiscountedCuCashflow,  //Extract to an other scope with month in the identity to avoid Zip?
                                              (nominal, discountedCumulated) => Math.Abs(discountedCumulated) >= Precision 
                                                                                          ? 1 - nominal / discountedCumulated 
                                                                                          : 0).ToArray(),
                _ => Enumerable.Empty<double>().ToArray(),
                };
}

For a certain projection period - defined by the Shift, $S$, and the Time-Step, $TS$, parameters - the Amortization Factor is then given by the product of the corresponding monthly amortization factors:

$$
AF = 1 - \prod _{i = S}^{S + TS - 1} \text{Monthly }AF_i ~.
$$

Each GIC will have his own AF.

In order to run off the business of a given Group of Contract one should provide a cash flow of 0s for the AoC Step with AoC Type CL and Novelty C. When computing the AF this results in the product of the monthly amortization factors of the period to be 1. In this case, the computed AF does not follow the formula above but is 1 allowing for the full release of the Technical Margin in the AM AoC Step.

In [0]:
public interface CurrentPeriodAmortizationFactor : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
        builder.ForScope<CurrentPeriodAmortizationFactor>(s => 
                 s.WithApplicability<AmfFromIfrsVariable>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow
                                                            || x.GetStorage().IsSecondaryScope(x.Identity.DataNode)));
                                                            
    private int shift => GetStorage().GetShift(0);//Identity.ProjectionPeriod
    private int timeStep => GetStorage().GetTimeStep(0);//Identity.ProjectionPeriod
    private double amortizedFactor => GetScope<MonthlyAmortizationFactorCashflow>(Identity)
                            .MonthlyAmortizationFactors
                            .Skip(shift)
                            .Take(timeStep)
                            .Aggregate(1d, (x, y) => x * y);
    [NotVisible] string EconomicBasis => GetContext();
    
    string EstimateType => EstimateTypes.F;
    double Value => Math.Abs(amortizedFactor - 1d) > Precision
                        ? 1d - amortizedFactor
                        : 1d;
}

public interface AmfFromIfrsVariable : CurrentPeriodAmortizationFactor
{
    double CurrentPeriodAmortizationFactor.Value => GetStorage().GetValue(Identity, (string)null, EstimateType, EconomicBasis, (int?)null);
}

<a id='actual-values'></a>
## Actual Values

We consider 4 types of Actual values, which are distinguished through their [Estimate Type](../DataModel/DataStructure#estimate-type):
- [Actuals](#actuals) (A)
- [Advance Actuals](#advance-actuals) (AA)
- [Overdue Actuals](#overdue-actuals) (OA)
- [Deferrable Actuals](#deferrable-actuals) (DA)

with the Estimate Type's system name shown between parenthesis above.

The following simplified AoC Chain applies for Advance and Overdue Actuals:
<br>&emsp;BOP
<br>&emsp;Release
<br>&emsp;Write-off
<br>&emsp;EOP

<a id='actual-base'></a>
### Actual Base

The Actual Base sets values for actual, advance, and overdue as follows:

$$
\text{Actual Base} (\text{AoC step}) = \left\{
\begin{array}{rl}
0 ~, & \text{if AoC Step's AoC Type = AM} \\
\text{Actual Base}(\rm{BOP}) + \text{Actual Base}(\rm{CF}) + \text{Actual Base}(\rm{WO}) ~, & \text{if AoC Step's AoC Type = EOP and EstimateType is not A} \\
\text{Imported Actual} ~, & \text{otherwise}
\end{array}
\right.
$$

where the value is also function of the [Estimate Type](../DataModel/DataStructure#EstimateType) and [Amount Type](../DataModel/DataStructure#AmountType), and the $\text{Imported Actual}$ value is described [here]().

In [0]:
public interface ActualBase : IScope<(ImportIdentity Id, string AmountType, string EstimateType, int? AccidentYear), ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<ActualBase>(s => s.WithApplicability<EmptyValuesActual>(x => x.GetStorage().ImportFormat == ImportFormats.Actual 
                                                                                       && !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode)
                                                                                       && x.Identity.Id.AocType == AocTypes.AM)
                                               .WithApplicability<EndOfPeriodActual>(x => x.GetStorage().ImportFormat != ImportFormats.Cashflow 
                                                                                       && !x.GetStorage().IsSecondaryScope(x.Identity.Id.DataNode)
                                                                                       && x.Identity.Id.AocType == AocTypes.EOP 
                                                                                       && x.Identity.EstimateType != EstimateTypes.A)
                                       );
    public double Value => GetStorage().GetValue(Identity.Id, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear); 
}

public interface EndOfPeriodActual : ActualBase
{
    double ActualBase.Value => GetScope<PreviousAocSteps>((Identity.Id, InputSource.Actual)).Values
                                        .Sum(aocStep => GetScope<ActualBase>((Identity.Id with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}, Identity.AmountType, Identity.EstimateType, Identity.AccidentYear)).Value);
}

public interface EmptyValuesActual : ActualBase
{
    double ActualBase.Value => 0;
}

<a id='actuals'></a>
### Actuals
The Actuals correspond to ActualBase values with estimate type $A$.
The only valid AoC Step is Release:

$$
\text{Actual} (\text{Release}) = \text{Actual Base} (\text{Release})|_{\text{Estimate Type} = A}
$$

In [0]:
public interface Actual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.A;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

<a id='advance-actuals'></a>
### Advance Actuals

Advance Actuals are cash flows with due date inside the reporting period but occured *before* the reporting period - They include:
- Receivable Claims
- Receivable Expenses
- Payable Premiums

Advance Actuals are given by

$$
\text{Advance Actual} (\text{AoC step}) = \text{Actual Base} (\text{AoC step})|_{\text{Estimate Type} = AA}
$$

In [0]:
public interface AdvanceActual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.AA;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

<a id='overdue-actuals'></a>
### Overdue Actuals

Overdue Actuals are cash flows with due date inside the reporting period but occur *after* the reporting period - They contain:
- Payable Claims
- Payable Expenses
- Receivable Premiums

The sign convention is the inverse of the default behavior - In particular: Premiums have positive value, whereas Claims and Expenses have negative value.

The Overdue Actuals are given by

$$
\text{Overdue Actual} (\text{AoC step}) = \text{Actual Base} (\text{AoC step})|_{\text{Estimate Type} = OA}
$$

In [0]:
public interface OverdueActual : IScope<ImportIdentity, ImportStorage>
{
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.OA;
    
    private int?[] accidentYears => GetStorage().GetAccidentYears(Identity.DataNode).ToArray();
    
    [NotVisible]
    ActualBase[] Actuals => GetScope<ValidAmountType>(Identity.DataNode).ActualAmountTypes
                                                            .SelectMany(at => accidentYears
                                                                .Select(ay => GetScope<ActualBase>((Identity, at, EstimateType, ay)))).ToArray();
}

<a id='deferrable-actuals'></a>
### Deferrable Actuals

Finally, Deferrable Actuals are given by

$$
\text{Deferrable Actual} (\text{AoC step}) = \left\{
\begin{array}{rl}
\sum_{\text{Amount Type}~\in~\text{\{ACA, AEA}\}}\text{Actual Base}(\rm{CF})|_{\text{Estimate Type = A}} ~, & \text{if AoC Step's AoC Type = CF } \\
- \text{Amortization Factor} \cdot \big(  \text{Deferrable Actual}(\rm{BOP}) + \text{Deferrable Actual}(\rm{CF}) \big) ~, & \text{if AoC Step's AoC Type = AM } \\
\text{Deferrable Actual}(\rm{BOP}) + \text{Deferrable Actual}(\rm{CF}) + \text{Deferrable Actual}(\rm{AM}) ~, & \text{if AoC Step's AoC Type = EOP } \\
\text{Input Actual}|_{\text{Estimate Type = DA}} ~, & \text{ otherwise } \\
\end{array}
\right.
$$

where $ACA$ and $AEA$ are *Aquisition* Amount Types from **Attributable Commission** and **Attributable Expenses**, respectively.

In [0]:
public interface DeferrableActual : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<DeferrableActual>(s => s.WithApplicability<DeferrableActualForCurrentBasis>(x => x.Identity.ValuationApproach == ValuationApproaches.VFA, 
                                                                                                              p => p.ForMember(s => s.EconomicBasis))
                                                     .WithApplicability<ReleaseDeferrable>(x => x.Identity.AocType == AocTypes.CF)
                                                     .WithApplicability<AmortizationDeferrable>(x => x.Identity.AocType == AocTypes.AM)
                                                     .WithApplicability<EndOfPeriodDeferrable>(x => x.Identity.AocType == AocTypes.EOP)
                                       );
    
    [IdentityProperty][NotVisible][Dimension(typeof(EstimateType))]
    string EstimateType => EstimateTypes.DA;
    
    [NotVisible] string EconomicBasis => EconomicBases.L;
        
    public double Value => GetStorage().GetValue(Identity, (string)null, EstimateType, (int?)null);
}

public interface DeferrableActualForCurrentBasis : DeferrableActual
{
    [NotVisible] string DeferrableActual.EconomicBasis => EconomicBases.C;
}

public interface ReleaseDeferrable : DeferrableActual
{
    double DeferrableActual.Value => GetStorage().GetAttributableExpenseAndCommissionAmountType().Sum(at => GetScope<ActualBase>((Identity, at, EstimateTypes.A, (int?)null)).Value);
}

public interface AmortizationDeferrable : DeferrableActual
{
    private double AmortizationFactor => GetScope<CurrentPeriodAmortizationFactor>(Identity, o => o.WithContext(EconomicBasis)).Value;
    private double AggregatedDeferrable => GetScope<PreviousAocSteps>((Identity, InputSource.Actual)).Values
                                            .Sum(aocStep => GetScope<DeferrableActual>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
    double DeferrableActual.Value => -1d * AggregatedDeferrable * AmortizationFactor;
}

public interface EndOfPeriodDeferrable : DeferrableActual
{
    double DeferrableActual.Value => GetScope<PreviousAocSteps>((Identity, InputSource.Actual)).Values
                                        .Sum(aocStep => GetScope<DeferrableActual>(Identity with {AocType = aocStep.AocType, Novelty = aocStep.Novelty}).Value);
}

<a id="csm"></a>
## Contractual Service Margin, Loss Component and Loss Recovery Component

The Contractual Service Margin ($CSM$) denotes the unearned profit from an insurance contract or group of insurance contracts and plays a critical role in the calculation of profit & loss values. Similarly, the unearned loss is denoted by Loss Component ($LC$), Loss Recovery Component in the case of reinsurance contracts.

## Experience Adjustment on Premium

The Experience Adjustment (EA) on Premiums weights the cash-flow (CF aocSteps) for premium amount types by the PremiumAllocationFactor input for each group of insurance contract. 
The contributions of present values and actuals are computed separately.

$$
EA (\rm{PV}) = \text{Premium Allocation Factor} \cdot \big( PV (\text{AoC Type = CF}) \big)~, \\
$$
$$
EA (\text{Actual}) = \text{Premium Allocation Factor} \cdot \big( \text{Actual}(\text{AoC Type = CF}) \big) ~,
$$
where amount type premium and its children are considered, novelties in-force and new business are considered for $PV$ whereas novelty combined is considered for Actual.  The allocation is always done in the finest granularity (novelty, line of business, ..) possible.

In [0]:
public interface BeExperienceAdjustmentForPremium : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<BeExperienceAdjustmentForPremium>(s => s
                .WithApplicability<DefaultValueBeExperienceAdjustmentForPremium>(x => x.Identity.AocType != AocTypes.CF));

    string EstimateType => EstimateTypes.BEPA;
    string EconomicBasis => EconomicBases.L;
    
    PresentValue[] ByAmountType => GetStorage().GetPremiums().Select(pr => 
                                       Multiply( GetStorage().GetPremiumAllocationFactor(Identity), 
                                                 GetScope<PresentValue>((Identity, pr, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)) ) ).ToArray();                            
}

public interface DefaultValueBeExperienceAdjustmentForPremium : BeExperienceAdjustmentForPremium
{
    PresentValue[] BeExperienceAdjustmentForPremium.ByAmountType => Enumerable.Empty<PresentValue>().ToArray();
}

In [0]:
public interface ActualExperienceAdjustmentOnPremium : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) =>
            builder.ForScope<ActualExperienceAdjustmentOnPremium>(s => s
                .WithApplicability<DefaultValueActualExperienceAdjustmentOnPremium>(x => x.Identity.AocType != AocTypes.CF));
    
    public ActualBase[] ByAmountTypeAndEstimateType => GetStorage().GetPremiums().Select(pr =>
                                        Multiply( GetStorage().GetPremiumAllocationFactor(Identity),
                                                GetScope<ActualBase>((Identity, pr, EstimateTypes.A, (int?)null))) ).ToArray();
}

public interface DefaultValueActualExperienceAdjustmentOnPremium : ActualExperienceAdjustmentOnPremium
{
    ActualBase[] ActualExperienceAdjustmentOnPremium.ByAmountTypeAndEstimateType => Enumerable.Empty<ActualBase>().ToArray();
}

<a id="technical-margin"></a>
## Technical Margin

For the computation of the $CSM$ or $LC$ component values for each step of the [AoC Step Structure](#aoc-step-structure), it is convenient to introduce the 
notion of technical margin $TM$. In the case of BBA methodology, the Locked-in interest rates discounting is applied to obtain the PV and RA values.

This can be summarized as follows:

$$
TM(s) = \left\{
\begin{array}{rl}
TM(\rm{EOP}) \text{ of the previous period} ~ & \text{if }s = \text{BOP and Novelty is In-Force}.
\\
0 ~ & \text{if }s = \text{CF}.
\\
IAF \cdot~\text{Aggregated}~TM(\text{IA}) ~ & \text{if }s = \text{IA and Novelty is In-Force}.
\\
\rm{Premiums} + \text{Attributable Expense and Commissions} + \text{Investment Claims} ~ & \text{if }s = \text{EA}.
\\
-AF \cdot~\text{Aggregated}~TM(\text{AM})~ & \text{if }s = \text{AM}.
\\
\text{PV}(\text{s})\bigg|_{\substack{\text{Non Attributable} \\ \text{Amount Types} \\ \text{excluded}}}  
+ \text{RA}(\text{s})~ & \text{otherwise} 
\end{array}
\right.
$$

where

$$
\text{Aggregated}~TM (\text{AoC step}) = \sum_{s\in\text{ previous AoC steps}} TM(s)~.
$$

and the Interest Accretion Factor ($IAF$) is given by

$$
IAF = \prod_{i=1}^{TS} \text{Interest}_i - 1
$$

where $\text{Interest}_i$ is the monthly interest factor obtained from the [Yield Curve](#yield-curves) and $TS$ is the Time-Step.

Finally, the Premiums, Attributable Expense and Commissions and Investment Claims terms are given by:

$$
\rm{Premiums} = \sum_{\text{Amount Type}\in\{\text{PR and its children}\}}
EA(\rm{PV}) - EA(\text{Actual}) ~,
$$

$$
\text{Attributable Expense and Commissions} = \sum_{\text{Amount Type}\in\{\rm{ACA}, \rm{AEA}\}}
\big(PV_{\text{Novelty = I}} + PV_{\text{Novelty = N}} \big) - \text{Actual}_{\text{Novelty=C}} ~,
$$

$$
\text{Investment Claims } = \sum_{\text{Amount Type}\in\{\text{ICO and its children}\}}
\big(PV_{\text{Novelty = I}} + PV_{\text{Novelty = N}} \big) -  \text{Actual}_{\text{Novelty=C}}   ~,
$$

where the AoC Step **CF** is implicit for all formulas, $PV$ is the [present value](#present-value) with Estimate Type **BE** (Best Estimate), and Actuals have Estimate Types **A** (see details [here](#actual-values)). Note that according to the applied sign convention, the sign of the CF AoC Step of Best Estimate Present Value and Actual is the same, accounting for the minus sign in the formula. 

In [0]:
public interface TechnicalMargin : IScope<ImportIdentity, ImportStorage>
{
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) => 
        builder.ForScope<TechnicalMargin>(s => s.WithApplicability<TechnicalMarginForCurrentBasis>(x => x.Identity.ValuationApproach == ValuationApproaches.VFA, p => p.ForMember(s => s.EconomicBasis))
                                               .WithApplicability<TechnicalMarginForBOP>(x => x.Identity.AocType == AocTypes.BOP && x.Identity.Novelty == Novelties.I)
                                               .WithApplicability<TechnicalMarginDefaultValue>(x => x.Identity.AocType == AocTypes.CF)
                                               .WithApplicability<TechnicalMarginForIA>(x => x.Identity.AocType == AocTypes.IA && x.Identity.Novelty == Novelties.I)
                                               .WithApplicability<TechnicalMarginForEA>(x => x.Identity.AocType == AocTypes.EA && !x.Identity.IsReinsurance)
                                               .WithApplicability<TechnicalMarginForAM>(x => x.Identity.AocType == AocTypes.AM)
                                               );
    
    [NotVisible] string EconomicBasis => EconomicBases.L;
    
    double Value => GetScope<ValidAmountType>(Identity.DataNode).BeAmountTypes
                       .Except(GetStorage().GetNonAttributableAmountType())
                       .Sum(at => GetScope<PresentValue>((Identity, at, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value) +
                    GetScope<RaLocked>(Identity).Value;
                    
    double AggregatedValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                .Sum(aoc => GetScope<TechnicalMargin>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
}

public interface TechnicalMarginForCurrentBasis : TechnicalMargin
{
    [NotVisible] string TechnicalMargin.EconomicBasis => EconomicBases.C;
}

public interface TechnicalMarginForBOP : TechnicalMargin
{
    private double ValueCsm => GetStorage().GetValue(Identity, null, EstimateTypes.C, null);
    private double ValueLc => GetStorage().GetValue(Identity, null, EstimateTypes.L, null);
    private double ValueLr => GetStorage().GetValue(Identity, null, EstimateTypes.LR, null);
    
    double TechnicalMargin.Value => -1d * ValueCsm + ValueLc + ValueLr;
}

public interface TechnicalMarginDefaultValue : TechnicalMargin
{
    double TechnicalMargin.Value => default;
}

public interface TechnicalMarginForIA : TechnicalMargin
{
    private int timeStep => GetStorage().GetTimeStep(0);//Identity.Id.ProjectionPeriod
    private int shift => GetStorage().GetShift(0);//Identity.Id.ProjectionPeriod
    
    private double[] monthlyInterestFactor => GetScope<MonthlyRate>(Identity, o => o.WithContext(EconomicBasis)).Interest;
    
    private double interestAccretionFactor => Enumerable.Range(shift,timeStep)
                                                    .Select(i => GetElementOrDefault(monthlyInterestFactor, i/12))
                                                    .Aggregate(1d, (x, y) => x * y ) - 1d;
    
    double TechnicalMargin.Value => AggregatedValue * interestAccretionFactor;
}

public interface TechnicalMarginForEA : TechnicalMargin
{
    static ApplicabilityBuilder ScopeApplicabilityBuilderInner(ApplicabilityBuilder builder) => 
        builder.ForScope<TechnicalMargin>(s => s.WithApplicability<TechnicalMarginDefaultValue>(x => x.Identity.IsReinsurance));
                                               
    private string referenceAocType => GetScope<ReferenceAocStep>(Identity).Value.AocType;
    
    private double premiums => GetStorage().GetNovelties(referenceAocType, InputSource.Cashflow)
                                    .Sum(n => GetScope<BeExperienceAdjustmentForPremium>(Identity with {AocType = referenceAocType, Novelty = n}).ByAmountType.Sum(sc => sc.Value)) -
                                GetScope<ActualExperienceAdjustmentOnPremium>(Identity with {AocType = referenceAocType, Novelty = Novelties.C}).ByAmountTypeAndEstimateType.Sum(sc => sc.Value);
                           
    private double attributableExpenseAndCommissions => GetStorage().GetAttributableExpenseAndCommissionAmountType().Sum(d =>
                                                         GetStorage().GetNovelties(referenceAocType, InputSource.Cashflow)
                                                                     .Sum(n => GetScope<PresentValue>((Identity with {AocType = referenceAocType, Novelty = n}, d, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value) -
                                                         GetScope<ActualBase>((Identity with {AocType = referenceAocType, Novelty = Novelties.C}, d, EstimateTypes.A, (int?)null)).Value);

    private double investmentClaims => GetStorage().GetInvestmentClaims().Sum(ic =>
                                         GetStorage().GetNovelties(referenceAocType, InputSource.Cashflow)
                                                     .Sum(n => GetScope<PresentValue>((Identity with {AocType = referenceAocType, Novelty = n}, ic, EstimateTypes.BE, (int?)null), o => o.WithContext(EconomicBasis)).Value) -
                                         GetScope<ActualBase>((Identity with {AocType = referenceAocType, Novelty = Novelties.C}, ic, EstimateTypes.A, (int?)null)).Value);
    
    double TechnicalMargin.Value => premiums + attributableExpenseAndCommissions + investmentClaims;
}

public interface TechnicalMarginForAM : TechnicalMargin
{
    double TechnicalMargin.Value => -1d * AggregatedValue * GetScope<CurrentPeriodAmortizationFactor>(Identity, o => o.WithContext(EconomicBasis)).Value;
}

<a id="switch-logic"></a>
## Switch Logic
    
The $CSM$ and $LC$ components are computed from the [technical margin](#technical-margin). In particular, for a given AoC step $s$, TM is allocated to the $LC$ when $\text{Aggregated } TM$ is positive whereas it is allocated to the $CSM$ when $\text{Aggregated } TM$ is negative:

$$
\begin{array}{rl}
CSM({\text{AoC Step}}) = 0,~~ LC({\text{AoC step}}) = TM({\text{AoC step}}) ~ & \text{if }\text{Aggregated }TM({\text{AoC step}}) > 0.
\\
CSM({\text{AoC Step}}) = - TM({\text{AoC step}}),~~ LC({\text{AoC step}}) = 0 ~ & \text{otherwise} 
\end{array}
$$


The figures reported under CSM are the opposite of the TM value in order to satisfy our sign convention.

It is possible to switch from $CSM$ to $LC$ and from $LC$ to $CSM$ at any AoC step $s$ with the only exception of **Amortization** where there is no switch from the previous step.
When a switch occurs the total contribution to the $CSM$ ($LC$) prior the switching step is brought to 0 and the remaing amount is allocated to $LC$ ($CSM$).

<a id="gross-case"></a>
### Gross case (i.e. no reinsurance)

The switch logic is applied ***separately*** to the In-Force and New Business novelties. The Combined Liability AoC Step **CL** will bring both contributions to CSM and LC together as the novelities are summed up.

In detail, and as we go through the AoC steps in the AoC chain, we have

**A)** For the **BOP**:
$$
\begin{array}{rl}
CSM(\text{BOP}) &= CSM(\text{EOP}) \text{ of the previous period, for Novelty In-Force}
\\
LC(\text{BOP}) &= LC(\text{EOP}) \text{ of the previous period, for Novelty In-Force}
\end{array}
$$


**B)** For the **following AoC steps**, the switch logic is preferably formulated using the delta variations between steps, $\Delta CSM$ and $\Delta LC$ for the CSM and the LC respectively.

The switch logic for these AoC steps with the *exception* of Combined Liability is:

If $\text{Aggregated }TM(\text{AoC Step}) > 0$ and $\text{Aggregated }TM(\text{AoC Step}) + TM(\text{AoC Step}) \le 0$, then there is a switch from $LC$ to $CSM$:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= -TM(\text{AoC Step}) -\text{Aggregated } TM(\text{AoC Step})
\\
\Delta LC(\text{AoC step}) &= -\text{Aggregated } TM(\text{AoC Step}) 
\end{array}
$$

On the other hand, if $\text{Aggregated }TM(\text{AoC Step}) \le 0$ and $\text{Aggregated }TM(\text{AoC Step}) + TM(\text{AoC Step}) > 0$, then the switch is from $CSM$ to $LC$:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= \text{Aggregated } TM(\text{AoC Step}) 
\\
\Delta LC(\text{AoC step}) &= TM(\text{AoC Step}) + \text{Aggregated } TM(\text{AoC Step}) 
\end{array}
$$

If there is no switch from $CSM$, i.e. $\text{Aggregated }TM(\text{AoC Step}) \le 0$ and $\text{Aggregated }TM(\text{AoC Step}) + TM(\text{AoC Step}) \le 0 $, then:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= -TM(\text{AoC Step})
\\
\Delta LC(\text{AoC step}) &= 0
\end{array}
$$

And finally, if there is no switch from $LC$, i.e. $\text{Aggregated }TM(\text{AoC Step}) > 0$ and $\text{Aggregated }TM(\text{AoC Step}) + TM(\text{AoC Step}) \ge 0 $, then:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= 0
\\
\Delta LC(\text{AoC step}) &= TM(\text{AoC Step})
\end{array}
$$

**C)** For the AoC step Combined Liability **CL**, the switch logic is similar to the one above, except that a switch from $LC$ to $CSM$ can happen **because of New Business and despite of In-Force**, in which case we have:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= -TM(\text{AoC Step}) -\big(\text{Aggregated } TM(\text{Last In-Force AoC step})+TM(\text{Last In-Force AoC step}) \big)
\\
\Delta LC(\text{AoC step}) &= -\big(\text{Aggregated } TM(\text{Last In-Force AoC step}) + TM(\text{Last In-Force AoC step}) \big)
\end{array}
$$

If, on the other hand, the switch from $LC$ to $CSM$ happens **because of In-Force and despite of New Business**, then we have:

$$
\begin{array}{rl}
\Delta CSM(\text{AoC step}) &= -TM(\text{AoC Step}) -\big(\text{Aggregated } TM(\text{Last NB AoC step})+TM(\text{Last NB AoC step}) \big)
\\
\Delta LC(\text{AoC step}) &= -\big(\text{Aggregated } TM(\text{Last NB AoC step}) + TM(\text{Last NB AoC step}) \big)
\end{array}
$$

For the switch in the other direction, i.e. from $CSM$ to $LC$ the formulas are similar except that LC and CSM are swapped.

**D)** Finally, for the last AoC step, the **EOP**, we have:

$$
\begin{array}{rl}
CSM(\text{EOP}) &= \sum_{s~\in~\text{previous AoC steps}}\Delta CSM(s)
\\
LC(\text{EOP}) &= \sum_{s~\in~\text{previous AoC steps}}\Delta LC(s)
\end{array}
$$

<a id="reinsurance-case"></a>
### Reinsurance case

For the reinsurance case, the switch logic is identical to the one described above, except that it uses the gross Technical Margins multiplied by the weights  coming from the Reinsurance Coverage Parameter (see [Data Node Parameters](../DataModel/DataStructure#data-node-parameters)). In case a GRIC has multiple GICs, then these weighted $TM$s are aggregated.

In [0]:
public interface AllocateTechnicalMargin: IScope<ImportIdentity, ImportStorage>
{
    //Switch
    static ApplicabilityBuilder ScopeApplicabilityBuilder(ApplicabilityBuilder builder) => 
        builder.ForScope<AllocateTechnicalMargin>(s => s
                                             .WithApplicability<AllocateTechnicalMarginForReinsuranceCL>(x => x.Identity.IsReinsurance && x.Identity.AocType == AocTypes.CL) 
                                             .WithApplicability<AllocateTechnicalMarginForReinsurance>(x => x.Identity.IsReinsurance, 
                                                                                                       p => p.ForMember(s => s.ComputedEstimateType)
                                                                                                             .ForMember(s => s.HasSwitch))
                                             .WithApplicability<ComputeAllocateTechnicalMarginWithIfrsVariable>(x => x.GetStorage().IsSecondaryScope(x.Identity.DataNode))                                             
                                             .WithApplicability<AllocateTechnicalMarginForBop>(x => x.Identity.AocType == AocTypes.BOP)
                                             .WithApplicability<AllocateTechnicalMarginForCl>(x => x.Identity.AocType == AocTypes.CL)
                                             .WithApplicability<AllocateTechnicalMarginForEop>(x => x.Identity.AocType == AocTypes.EOP)
                                             );
    
    [NotVisible] double AggregatedTechnicalMargin => GetScope<TechnicalMargin>(Identity).AggregatedValue; 
    [NotVisible] double TechnicalMargin => GetScope<TechnicalMargin>(Identity).Value;
    
    [NotVisible] string ComputedEstimateType => ComputeEstimateType(GetScope<TechnicalMargin>(Identity).AggregatedValue + TechnicalMargin);
    [NotVisible] bool HasSwitch => ComputedEstimateType != ComputeEstimateType(GetScope<TechnicalMargin>(Identity).AggregatedValue);
     
    //Allocate   
    [NotVisible] string EstimateType => GetContext();
    
    double Value => (HasSwitch, EstimateType == ComputedEstimateType) switch {
            (true, true) => TechnicalMargin + AggregatedTechnicalMargin,
            (true, false) => -1d * AggregatedTechnicalMargin,
            (false, true) => TechnicalMargin,
            _ => default
        };
    
    string ComputeEstimateType(double aggregatedTechnicalMargin) => aggregatedTechnicalMargin > Precision ? EstimateTypes.L : EstimateTypes.C;
}

public interface ComputeAllocateTechnicalMarginWithIfrsVariable : AllocateTechnicalMargin
{                                  
    double AllocateTechnicalMargin.TechnicalMargin => ComputeTechnicalMarginFromIfrsVariables(Identity);
    double AllocateTechnicalMargin.AggregatedTechnicalMargin => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                                                    .Sum(aoc => ComputeTechnicalMarginFromIfrsVariables(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}));
                                                                    
    private double ComputeTechnicalMarginFromIfrsVariables(ImportIdentity id)
    {
        return GetStorage().GetValue(Identity, null, EstimateTypes.LR, null) +
               GetStorage().GetValue(Identity, null, EstimateTypes.L, null) - 
               GetStorage().GetValue(Identity, null, EstimateTypes.C, null);
    }
}

public interface AllocateTechnicalMarginForReinsurance : AllocateTechnicalMargin
{  
   //TODO add Reinsurance Coverage Update (RCU, Novelty=I) AocStep
    private IEnumerable<string> underlyingGic => GetStorage().GetUnderlyingGic(Identity);
   
    private double weightedUnderlyingTM => underlyingGic.Sum(gic => GetStorage().GetReinsuranceCoverage(Identity, gic) * 
                                                                      GetScope<AllocateTechnicalMargin>(Identity with {DataNode = gic}).TechnicalMargin);
                                                                      
    private double weightedUnderlyingAggregatedTM => underlyingGic.Sum(gic => GetStorage().GetReinsuranceCoverage(Identity, gic) * 
                                                                      GetScope<AllocateTechnicalMargin>(Identity with {DataNode = gic}).AggregatedTechnicalMargin);
    
    private string ComputeReinsuranceEstimateType(double aggregatedFcf) => aggregatedFcf > Precision ? EstimateTypes.LR : EstimateTypes.C;
    
    [NotVisible] string AllocateTechnicalMargin.ComputedEstimateType => ComputeReinsuranceEstimateType(weightedUnderlyingAggregatedTM + weightedUnderlyingTM);
    [NotVisible] bool AllocateTechnicalMargin.HasSwitch => ComputedEstimateType != ComputeReinsuranceEstimateType(weightedUnderlyingAggregatedTM); 
}

public interface AllocateTechnicalMarginForReinsuranceCL : AllocateTechnicalMargin
{   
    //In common1
    private IEnumerable<string> underlyingGic => GetStorage().GetUnderlyingGic(Identity);
   
    private double weightedUnderlyingTM => underlyingGic.Sum(gic => GetStorage().GetReinsuranceCoverage(Identity, gic) * 
                                                                      GetScope<AllocateTechnicalMargin>(Identity with {DataNode = gic}).TechnicalMargin);
                                                                      
    private double weightedUnderlyingAggregatedTM => underlyingGic.Sum(gic => GetStorage().GetReinsuranceCoverage(Identity, gic) * 
                                                                      GetScope<AllocateTechnicalMargin>(Identity with {DataNode = gic}).AggregatedTechnicalMargin);
    private string ComputeReinsuranceEstimateType(double aggregatedFcf) => aggregatedFcf > Precision ? EstimateTypes.LR : EstimateTypes.C;
    
    string AllocateTechnicalMargin.ComputedEstimateType => ComputeReinsuranceEstimateType(weightedUnderlyingAggregatedTM + weightedUnderlyingTM);
     //In common2
    private double balancingValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow))
                                        .Values
                                        .GroupBy(x => x.Novelty)
                                        .Select(g => g.Last())
                                        .Sum(aoc => { 
                                            var id = Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty};
                                            return GetScope<AllocateTechnicalMargin>(id).ComputedEstimateType != ComputedEstimateType ? 
                                                   GetScope<AllocateTechnicalMargin>(id).TechnicalMargin + GetScope<AllocateTechnicalMargin>(id).AggregatedTechnicalMargin
                                                   : (double)default; });
                                                   
    [NotVisible] bool AllocateTechnicalMargin.HasSwitch => Math.Abs(balancingValue) > Precision;
    [NotVisible] double AllocateTechnicalMargin.AggregatedTechnicalMargin => balancingValue;    
}

public interface AllocateTechnicalMarginForCl : AllocateTechnicalMargin
{
    private double balancingValue => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow))
                                        .Values
                                        .GroupBy(x => x.Novelty)
                                        .Select(g => g.Last())
                                        .Sum(aoc => { 
                                            var id = Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty};
                                            return GetScope<AllocateTechnicalMargin>(id).ComputedEstimateType != ComputedEstimateType ? 
                                                   GetScope<AllocateTechnicalMargin>(id).TechnicalMargin + GetScope<AllocateTechnicalMargin>(id).AggregatedTechnicalMargin
                                                   : (double)default; });

    [NotVisible] bool AllocateTechnicalMargin.HasSwitch => Math.Abs(balancingValue) > Precision;
    
    [NotVisible] double AllocateTechnicalMargin.AggregatedTechnicalMargin => balancingValue;
}

public interface AllocateTechnicalMarginForBop : AllocateTechnicalMargin
{   
   bool AllocateTechnicalMargin.HasSwitch => false;
}

public interface AllocateTechnicalMarginForEop : AllocateTechnicalMargin
{
    double AllocateTechnicalMargin.Value  => GetScope<PreviousAocSteps>((Identity, InputSource.Cashflow)).Values
                                                .Sum(aoc => GetScope<AllocateTechnicalMargin>(Identity with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);
    [NotVisible] string AllocateTechnicalMargin.ComputedEstimateType => ComputeEstimateType(AggregatedTechnicalMargin);
}

For the **Group of Reinsurance Contracts** (GRICs), the switch is computed within the underlying GIC context and $CSM$ and $LR$ are then computed in the same manner as for the underlying GICs CSM and LC.

The scopes below are simply used to set the appropriate Estimate Type (C for $CSM$, L for $LC$ and LR for $LoReCo$), as well as to set $CSM$ values to be positive:

In [0]:
public interface ContractualServiceMargin : IScope<ImportIdentity, ImportStorage>
{
    [NotVisible]string EstimateType => EstimateTypes.C;
    
    double Value => -1d * GetScope<AllocateTechnicalMargin>(Identity, o => o.WithContext(EstimateType)).Value;
}

In [0]:
public interface LossComponent : IScope<ImportIdentity, ImportStorage>
{
    [NotVisible]string EstimateType => EstimateTypes.L;
    
    double Value => GetScope<AllocateTechnicalMargin>(Identity, o => o.WithContext(EstimateType)).Value;
}

In [0]:
public interface LossRecoveryComponent : IScope<ImportIdentity, ImportStorage>
{ 
    [NotVisible]string EstimateType => EstimateTypes.LR;
    
    double Value => GetScope<AllocateTechnicalMargin>(Identity, o => o.WithContext(EstimateType)).Value;
}

## Computing All Scopes

Since all the calculations defined in the various scopes are to be performed together, it is useful to introduce the scopes: <code>ComputeIfrsVarsActuals</code>, <code>ComputeIfrsVarsCashflows</code> and the <code>ComputeIfrsVarsOpenings</code> below, which given a certain Import Format and Import Storage, performs all the calculations (defined above) based on the corresponding data:

In [0]:
public interface PvToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> PvLocked => GetScope<PvLocked>(Identity).PresentValues.Where(x => Math.Abs(x.Value) >= Precision).Select(x => new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                                                                                                                                          EstimateType = x.Identity.EstimateType, 
                                                                                                                                          DataNode = x.Identity.Id.DataNode, 
                                                                                                                                          AocType = x.Identity.Id.AocType, 
                                                                                                                                          Novelty = x.Identity.Id.Novelty, 
                                                                                                                                          AccidentYear = x.Identity.AccidentYear,
                                                                                                                                          AmountType = x.Identity.AmountType,
                                                                                                                                          Value = x.Value,
                                                                                                                                          Partition = GetStorage().TargetPartition
                                                                                                                                        });
    IEnumerable<IfrsVariable> PvCurrent => GetScope<PvCurrent>(Identity).PresentValues.Where(x => Math.Abs(x.Value) >= Precision).Select(x => new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                                                                                                                                          EstimateType = x.Identity.EstimateType, 
                                                                                                                                          DataNode = x.Identity.Id.DataNode, 
                                                                                                                                          AocType = x.Identity.Id.AocType, 
                                                                                                                                          Novelty = x.Identity.Id.Novelty, 
                                                                                                                                          AccidentYear = x.Identity.AccidentYear,
                                                                                                                                          AmountType = x.Identity.AmountType,
                                                                                                                                          Value = x.Value,
                                                                                                                                          Partition = GetStorage().TargetPartition
                                                                                                                                        });
}

In [0]:
public interface RaToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> RaCurrent => GetScope<RaCurrent>(Identity).PresentValues.Where(x => Math.Abs(x.Value) >= Precision).Select(x => new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                                                                                                                                          EstimateType = x.Identity.EstimateType, 
                                                                                                                                          DataNode = x.Identity.Id.DataNode, 
                                                                                                                                          AocType = x.Identity.Id.AocType, 
                                                                                                                                          Novelty = x.Identity.Id.Novelty, 
                                                                                                                                          AccidentYear = x.Identity.AccidentYear,
                                                                                                                                          AmountType = null,
                                                                                                                                          Value = x.Value,
                                                                                                                                          Partition = GetStorage().TargetPartition
                                                                                                                                        });
    IEnumerable<IfrsVariable> RaLocked => GetScope<RaLocked>(Identity).PresentValues.Where(x => Math.Abs(x.Value) >= Precision).Select(x => new IfrsVariable{ EconomicBasis = x.EconomicBasis, 
                                                                                                                                          EstimateType = x.Identity.EstimateType, 
                                                                                                                                          DataNode = x.Identity.Id.DataNode, 
                                                                                                                                          AocType = x.Identity.Id.AocType, 
                                                                                                                                          Novelty = x.Identity.Id.Novelty, 
                                                                                                                                          AccidentYear = x.Identity.AccidentYear,
                                                                                                                                          AmountType = null,
                                                                                                                                          Value = x.Value,
                                                                                                                                          Partition = GetStorage().TargetPartition
                                                                                                                                        });
}

In [0]:
public interface ActualToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> Actual => GetScope<Actual>(Identity).Actuals.Where(x => Math.Abs(x.Value) >= Precision).Select(x => new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                                   DataNode = x.Identity.Id.DataNode,
                                   AocType = x.Identity.Id.AocType,
                                   Novelty = x.Identity.Id.Novelty,
                                   AccidentYear = x.Identity.AccidentYear,
                                   AmountType = x.Identity.AmountType,
                                   Value = x.Value,
                                   Partition = GetStorage().TargetPartition
                                });
    IEnumerable<IfrsVariable> AdvanceActual => GetScope<AdvanceActual>(Identity).Actuals.Where(x => Math.Abs(x.Value) >= Precision).Select(x => new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                                   DataNode = x.Identity.Id.DataNode,
                                   AocType = x.Identity.Id.AocType,
                                   Novelty = x.Identity.Id.Novelty,
                                   AccidentYear = x.Identity.AccidentYear,
                                   AmountType = x.Identity.AmountType,
                                   Value = x.Value,
                                   Partition = GetStorage().TargetPartition
                                });
    IEnumerable<IfrsVariable> OverdueActual => GetScope<OverdueActual>(Identity).Actuals.Where(x => Math.Abs(x.Value) >= Precision).Select(x => new IfrsVariable{ EstimateType = x.Identity.EstimateType,
                                   DataNode = x.Identity.Id.DataNode,
                                   AocType = x.Identity.Id.AocType,
                                   Novelty = x.Identity.Id.Novelty,
                                   AccidentYear = x.Identity.AccidentYear,
                                   AmountType = x.Identity.AmountType,
                                   Value = x.Value,
                                   Partition = GetStorage().TargetPartition
                                });
}

In [0]:
public interface DeferrableToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> DeferrableActual => GetScope<DeferrableActual>(Identity).RepeatOnce().Where(x => Math.Abs(x.Value) >= Precision).Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                    DataNode = x.Identity.DataNode,
                                    AocType = x.Identity.AocType,
                                    Novelty = x.Identity.Novelty,
                                    AccidentYear = null,
                                    Value = x.Value,
                                    Partition = GetStorage().TargetPartition
                                    });
}

In [0]:
public interface EaForPremiumToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    IEnumerable<IfrsVariable> BeEAForPremium => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
                                                || Identity.IsReinsurance
                            ? Enumerable.Empty<IfrsVariable>()
                            : GetScope<BeExperienceAdjustmentForPremium>(Identity)
                              .ByAmountType
                              .Where(x => Math.Abs(x.Value) >= Precision)
                              .Select(sc => new IfrsVariable{ EstimateType = GetScope<BeExperienceAdjustmentForPremium>(Identity).EstimateType, 
                                                              DataNode = sc.Identity.Id.DataNode, 
                                                              AocType = sc.Identity.Id.AocType, 
                                                              Novelty = sc.Identity.Id.Novelty, 
                                                              AccidentYear = sc.Identity.AccidentYear,
                                                              EconomicBasis = sc.EconomicBasis,
                                                              AmountType = sc.Identity.AmountType,
                                                              Value = sc.Value,
                                                              Partition = sc.GetStorage().TargetPartition });
    
    IEnumerable<IfrsVariable> ActEAForPremium => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
                                                 || Identity.IsReinsurance
                            ? Enumerable.Empty<IfrsVariable>()
                            : GetScope<ActualExperienceAdjustmentOnPremium>(Identity)
                                .ByAmountTypeAndEstimateType
                                .Where(x => Math.Abs(x.Value) >= Precision)
                                .Select(sc => new IfrsVariable{ EstimateType = GetStorage().ExperienceAdjustEstimateTypeMapping[sc.Identity.EstimateType], 
                                                 DataNode = sc.Identity.Id.DataNode, 
                                                 AocType = sc.Identity.Id.AocType, 
                                                 Novelty = sc.Identity.Id.Novelty, 
                                                 AccidentYear = sc.Identity.AccidentYear,
                                                 //EconomicBasis = scope.EconomicBasis,
                                                 AmountType = sc.Identity.AmountType,
                                                 Value = sc.Value,
                                                 Partition = GetStorage().TargetPartition });
}

In [0]:
public interface TmToIfrsVariable: IScope<ImportIdentity, ImportStorage>
{
    private string EconomicBasis => Identity.ValuationApproach == ValuationApproaches.VFA ? EconomicBases.C : EconomicBases.L;
    IEnumerable<IfrsVariable> AmortizationFactor => Identity.AocType == AocTypes.AM
                                                                            ? GetScope<CurrentPeriodAmortizationFactor>(Identity, o => o.WithContext(EconomicBasis))
                                                                                .RepeatOnce()
                                                                                .Where(x => Math.Abs(x.Value) >= Precision)
                                                                                .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                                                                                               DataNode = x.Identity.DataNode,
                                                                                                               AocType = x.Identity.AocType,
                                                                                                               Novelty = x.Identity.Novelty,
                                                                                                               EconomicBasis = x.EconomicBasis,
                                                                                                               Value = x.Value,
                                                                                                               Partition = GetStorage().TargetPartition
                                                                                                               })
                                                                            : Enumerable.Empty<IfrsVariable>();
    IEnumerable<IfrsVariable> Csms => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
                            ? Enumerable.Empty<IfrsVariable>()
                            : GetScope<ContractualServiceMargin>(Identity).RepeatOnce()
                              .Where(x => Math.Abs(x.Value) >= Precision)
                              .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                   DataNode = x.Identity.DataNode,
                                   AocType = x.Identity.AocType,
                                   Novelty = x.Identity.Novelty,
                                   Value = x.Value,
                                   Partition = GetStorage().TargetPartition
                                });

     IEnumerable<IfrsVariable> Loss => GetStorage().DataNodeDataBySystemName[Identity.DataNode].LiabilityType == LiabilityTypes.LIC
                             ? Enumerable.Empty<IfrsVariable>()
                             : Identity.IsReinsurance 
                                ? GetScope<LossRecoveryComponent>(Identity).RepeatOnce()
                                    .Where(x => Math.Abs(x.Value) >= Precision)
                                    .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                                                   DataNode = x.Identity.DataNode,
                                                                   AocType = x.Identity.AocType,
                                                                   Novelty = x.Identity.Novelty,
                                                                   Value = x.Value,
                                                                   Partition = GetStorage().TargetPartition
                                                                })
                                : GetScope<LossComponent>(Identity).RepeatOnce()
                                    .Where(x => Math.Abs(x.Value) >= Precision)
                                    .Select(x => new IfrsVariable{ EstimateType = x.EstimateType,
                                                                   DataNode = x.Identity.DataNode,
                                                                   AocType = x.Identity.AocType,
                                                                   Novelty = x.Identity.Novelty,
                                                                   Value = x.Value,
                                                                   Partition = GetStorage().TargetPartition
                                                                });
}

In [0]:
public interface ComputeIfrsVarsActuals : ActualToIfrsVariable, DeferrableToIfrsVariable, EaForPremiumToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => Actual.Concat(AdvanceActual)
                                                               .Concat(OverdueActual)
                                                               .Concat(ActEAForPremium)
                                                               .Concat(DeferrableActual)
                                                               .Concat(Csms)
                                                               .Concat(Loss);
}

In [0]:
public interface ComputeIfrsVarsCashflows : PvToIfrsVariable, RaToIfrsVariable, DeferrableToIfrsVariable, EaForPremiumToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => PvLocked.Concat(PvCurrent)
                                                                 .Concat(RaCurrent)
                                                                 .Concat(RaLocked)
                                                                 .Concat(AmortizationFactor)
                                                                 .Concat(BeEAForPremium)
                                                                 .Concat(DeferrableActual)
                                                                 .Concat(Csms)
                                                                 .Concat(Loss);
}

In [0]:
public interface ComputeIfrsVarsOpenings : ActualToIfrsVariable, DeferrableToIfrsVariable, TmToIfrsVariable
{
    IEnumerable<IfrsVariable> CalculatedIfrsVariables => AdvanceActual.Concat(OverdueActual)
                                                                      .Concat(DeferrableActual)
                                                                      .Concat(Csms)
                                                                      .Concat(Loss);
} 